In [2]:
import sys
sys.path.append('/usr/local/lib/python3.5/site-packages')
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import networkx as nx
import os

In [3]:
carpeta = (os.getcwd()+'/tc02Data/')

# Nombres de los archivos: yeast_(...).txt
archivos = ['Y2H','AP-MS','LIT','LIT_Reguly']

# Lista donde se van a ir agregando los grafos en el orden de los archivos
Gs = []

for j,archivo in enumerate(archivos):
    data = pd.read_csv(carpeta+'yeast_'+archivo+'.txt', sep='\t', header=None)
    
    G = nx.Graph()
    
    for i in range(len(data)):
        G.add_edges_from([(data[0][i],data[1][i])])
    Gs.append(G)

# El último archivo, LIT_Reguly, es el único que tiene encabezado
# Quise poner header automático pero devuelve un error, así que elimino lo que sobra a lo bruto
Gs[3].remove_node("Bait gene/protein")
Gs[3].remove_node("Hit gene/protein")

In [4]:
# Proteinas esenciales
data_ess = pd.read_csv(carpeta+'Essential_ORFs_paperHe.txt', sep='\t', header=0,skipfooter=4,usecols=[1])

# Para eliminar los espacios en los nombres de las proteinas
data_ess['ORF_name'] = data_ess['ORF_name'].map(lambda x: x.strip())

ess = data_ess["ORF_name"].tolist()
del ess[0] # como antes, elimino el encabezado

# ess es la lista de proteinas esenciales

C:\Users\Luli\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.
  


In [5]:
# Agrego la caracteristica de ser o no esencial

for G in Gs:
    nodos_G = set(G.nodes()) # set de nodos de G
    nodos_ess_G = nodos_G.intersection(set(ess)) # nodos esenciales de G (como interseccion entre nodos de G y esenciales)
    nodos_no_ess_G = nodos_G.difference(set(ess)) # nodos no esenciales de G (como diferencia entre nodos de G y esenciales)
    
    # Agrego el atributo correspondiente a cada nodo
    G.add_nodes_from(nodos_ess_G, essential=True)
    G.add_nodes_from(nodos_no_ess_G, essential=False)

# Para comprobarlo me fije que la cantidad de nodos sea la misma antes y despues de esto

In [6]:
# Alfa y beta calculados en los ajustes lineales
alpha = [0.0162, 0.0468, 0.0815, 0.0456]
beta = [0.1722, 0.181, 0.2437, 0.0733]
err_alpha = [0.0095, 0.0121, 0.0154, 0.0024]
err_beta = [0.0678, 0.1187, 0.0862, 0.0252]

In [7]:
# Para calcular los nodos del mismo tipo según el modelo lineal, usamos los alfa y beta calculados
# y además defino una probabilidad que depende del grado del nodo (k) y de la red (i)
def Pe(k,i):
    Pe = 1 - (1-beta[i]) * (1-alpha[i])**k
    err_Pe = np.sqrt( (1-alpha[i])**(2*k) * err_beta[i]**2 + (1-beta[i])**2 + k**2 * (1-alpha[i])**(2*(k-1)) * err_alpha[i]**2 )
    return (Pe, err_Pe)

In [8]:
pairs_total = []
pairs_sametype = []
pairs_exp = []
pairs_exp_err = []

for n,G in enumerate(Gs):
    nodos_lista = list(G.nodes())
    grados_dict = dict(G.degree())
    ess_dict = nx.get_node_attributes(G,'essential')

    A = nx.to_numpy_matrix(G) # matriz de adyacencia de G
    T = len(A) # tamaño de la matriz, debe ser igual que la cantidad de nodos, me aseguro:

    if G.number_of_nodes()!=T:
        print('El grafo y la matriz no son del mismo tamaño')

    for i in range(T):
        A[i,i]=0 # Como hay auto loops, pongo ceros en la diagonal

    # Creo la matriz de adyacencia al cuadrado
    A2 = A**2
    # El lugar i,j de esta matriz me dice cuantos caminos de longitud 2 hay entre el nodo i y el j

    # Como quiero quedarme con pares de nodos que tengan al menos 3 vecinos en común,
    # busco que haya al menos 3 caminos de longitud 2 entre ellos
    I,J = np.where(A2 >= 3)
    # obtengo los indices de los lugares
    
    # Cuento cantidad de pares de nodos con 3 o más vecinos en común
    pares = 0
    pares_iguales = 0
    N_exp = 0
    errPisquared = 0

    for i in range(len(I)):
        if I[i]!=J[i] and A[I[i],J[i]] == 0:
            pares +=1
            nodo1 = nodos_lista[I[i]]
            nodo2 = nodos_lista[J[i]]
            Pe1, errPe1 = Pe(grados_dict[nodo1],n)
            Pe2, errPe2 = Pe(grados_dict[nodo2],n)
            P1 = Pe1 * Pe2 + (1-Pe1) * (1-Pe2)
            errP1squared = ((2*Pe1-1)*errPe1)**2 + ((2*Pe2-1)*errPe2)**2
            errPisquared = errPisquared + errP1squared
            N_exp = N_exp + P1
            if ess_dict[nodo1] == ess_dict[nodo2]:
                pares_iguales +=1
    pares = int(pares/2)
    pares_iguales = int(pares_iguales/2)
    N_exp = int(N_exp/2)
    N_err = np.sqrt(errPisquared/2)

    pairs_total.append(pares)
    pairs_sametype.append(pares_iguales)
    pairs_exp.append(N_exp)
    pairs_exp_err.append(N_err)

print(pairs_total)
print(pairs_sametype)
print(pairs_exp)
print(pairs_exp_err)

[522, 11613, 730, 10777]
[352, 5907, 389, 6187]
[290, 7482, 396, 5660]
[11.594464790910704, 83.887685896795958, 11.435218862365005, 64.259100095426774]


In [9]:
# Para el caso de Y2H hay que considerar los pares que tengan al menos 1 vecino en común
G=Gs[0]
nodos_lista = list(G.nodes())
grados_dict = dict(G.degree())
ess_dict = nx.get_node_attributes(G,'essential')

A = nx.to_numpy_matrix(G) # matriz de adyacencia de G
T = len(A) # tamaño de la matriz, debe ser igual que la cantidad de nodos, me aseguro:

if G.number_of_nodes()!=T:
    print('El grafo y la matriz no son del mismo tamaño')

for i in range(T):
    A[i,i]=0 # Como hay auto loops, pongo ceros en la diagonal

# Creo la matriz de adyacencia al cuadrado
A2 = A**2
# El lugar i,j de esta matriz me dice cuantos caminos de longitud 2 hay entre el nodo i y el j

# Como quiero quedarme con pares de nodos que tengan al menos 1 vecino en común,
# busco que haya al menos 1 camino de longitud 2 entre ellos
I,J = np.where(A2 >= 1)
# obtengo los indices de los lugares

# Cuento cantidad de pares de nodos con 3 o más vecinos en común
pares = 0
pares_iguales = 0
N_exp = 0
errPisquared = 0

for i in range(len(I)):
    if I[i]!=J[i] and A[I[i],J[i]] == 0:
        pares +=1
        nodo1 = nodos_lista[I[i]]
        nodo2 = nodos_lista[J[i]]
        Pe1, errPe1 = Pe(grados_dict[nodo1],n)
        Pe2, errPe2 = Pe(grados_dict[nodo2],n)
        P1 = Pe1 * Pe2 + (1-Pe1) * (1-Pe2)
        errP1squared = ((2*Pe1-1)*errPe1)**2 + ((2*Pe2-1)*errPe2)**2
        errPisquared = errPisquared + errP1squared
        N_exp = N_exp + P1
        if ess_dict[nodo1] == ess_dict[nodo2]:
            pares_iguales +=1
pares = int(pares/2)
pares_iguales = int(pares_iguales/2)
N_exp = int(N_exp/2)
N_err = np.sqrt(errPisquared/2)

print(pares)
print(pares_iguales)
print(N_exp)
print(N_err)

23073
15087
13693
112.543088712
